<a href="https://colab.research.google.com/github/Albly/Deep_learning_MIMO_channel_denoise/blob/main/evaluate_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Albly/Deep_learning_MIMO_channel_denoise.git
%cd Deep_learning_MIMO_channel_denoise
!pip install mat73 -q

%load_ext autoreload
%autoreload 2

Cloning into 'Deep_learning_MIMO_channel_denoise'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 105 (delta 55), reused 74 (delta 30), pack-reused 0
Receiving objects: 100% (105/105), 2.46 MiB | 18.54 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/Deep_learning_MIMO_channel_denoise


In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import glob
import scipy

from src.mat_dataset import MAT_Dataset, gen_noise
from torch.utils.data import DataLoader
from torchsummary import summary
from src.models import Beam_DnCNN_3D, train_model
import json

from src.metrics import complex_MSE, complex_LOG_MSE

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
file_path = '/content/gdrive/MyDrive/ChannelQriga_Freq_5kmh_Scena0_test_MIR_GIT_RAW_NR_BERLIN_UMa_NLOS_3p5GHz_seed_1001.mat'
mode_path = glob.glob('./checkpoints/*.pt')

dataset = MAT_Dataset(path = file_path, UEs = [0,1,2,3])
dataloader = DataLoader(dataset, 5, shuffle = False)


In [12]:
cfg = json.load(open("model_config.txt"))
model = Beam_DnCNN_3D(cfg = cfg, n_layers = 15, n_features = 20)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights = torch.load(mode_path[0], map_location=torch.device('cpu'))
model.load_state_dict(weights)
model.to(device)
model.eval()

out = torch.tensor([], dtype = torch.complex64)

with torch.no_grad():
    for snr in range(-20, -4):
        out = torch.tensor([], dtype = torch.complex64).to(device)

        for signal in dataloader:
            noise = gen_noise(signal, snr)
            signal, noise = signal.to(device), noise.to(device)

            Power_noise = torch.sum(torch.abs(noise)**2 , dim = (1,2,3))
            sigma = Power_noise / (cfg['N_time']*cfg['N_Az']*cfg['N_El']*cfg['N_pol'])

            denoised_data, _, _ = model(signal + noise, norma = torch.sqrt(sigma))
            out = torch.concat((out, denoised_data ), dim = 0)
        out = out.to('cpu')
        name = 'denoised_5kmh_{0}_SNR.mat'.format(snr)
        scipy.io.savemat("/content/gdrive/MyDrive/outs/"+name, {"H_denoised": out.numpy()})


/content/Deep_learning_MIMO_channel_denoise/src/models.py:67: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data, a = 0, mode='fan_in')
